In [23]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

# Pegando os dados


In [25]:
df_vazao = pd.read_csv('/dados_vazao_agua.csv')
df_vazao.head()

,Unnamed: 0,vazao
0,2025-06-01 00:00:00,0.962513
1,2025-06-01 00:15:00,0.536287
2,2025-06-01 00:30:00,0.000000
3,2025-06-01 00:45:00,0.143800
4,2025-06-01 01:00:00,0.463535


# Tratando os dados

Colocando colunas para o treinamento

In [26]:
df_vazao['hora'] = df.index.hour
df_vazao['dia_semana'] = df.index.day_of_week

df_vazao['vazao_15min'] = df_vazao['vazao'].shift(1)
df_vazao['vazao_1h'] = df_vazao['vazao'].shift(4)
df_vazao['vazao_24h'] = df_vazao['vazao'].shift(96)

df_vazao

,Unnamed: 0,vazao,hora,dia_semana,vazao_15min,vazao_1h,vazao_24h
0,2025-06-01 00:00:00,0.962513,0,6,NaN,NaN,NaN
1,2025-06-01 00:15:00,0.536287,0,6,0.962513,NaN,NaN
2,2025-06-01 00:30:00,0.000000,0,6,0.536287,NaN,NaN
3,2025-06-01 00:45:00,0.143800,0,6,0.000000,NaN,NaN
4,2025-06-01 01:00:00,0.463535,1,6,0.143800,0.962513,NaN
...,...,...,...,...,...,...,...
11708,2025-09-30 23:00:00,0.372051,23,1,4.131517,3.306963,1.484080
11709,2025-09-30 23:15:00,1.139838,23,1,0.372051,4.312727,0.609547
11710,2025-09-30 23:30:00,0.880325,23,1,1.139838,3.753184,0.229752
11711,2025-09-30 23:45:00,1.326618,23,1,0.880325,4.131517,1.606732


Quantiade de dados nan

In [27]:
df_vazao.isna().sum()

,0
Unnamed: 0,0
vazao,0
hora,0
dia_semana,0
vazao_15min,1
vazao_1h,4
vazao_24h,96


Os dados nan vão ser retirados do dataset. Pois, essas amostras só apareceram quando a vazão de água na tubulução foi igual à zero

In [28]:
df_vazao = df_vazao.dropna()

In [29]:
df_vazao

,Unnamed: 0,vazao,hora,dia_semana,vazao_15min,vazao_1h,vazao_24h
96,2025-06-02 00:00:00,0.849757,0,0,1.428671,0.551178,0.962513
97,2025-06-02 00:15:00,1.298739,0,0,0.849757,1.873185,0.536287
98,2025-06-02 00:30:00,0.298227,0,0,1.298739,1.405025,0.000000
99,2025-06-02 00:45:00,0.000000,0,0,0.298227,1.428671,0.143800
100,2025-06-02 01:00:00,0.647341,1,0,0.000000,0.849757,0.463535
...,...,...,...,...,...,...,...
11708,2025-09-30 23:00:00,0.372051,23,1,4.131517,3.306963,1.484080
11709,2025-09-30 23:15:00,1.139838,23,1,0.372051,4.312727,0.609547
11710,2025-09-30 23:30:00,0.880325,23,1,1.139838,3.753184,0.229752
11711,2025-09-30 23:45:00,1.326618,23,1,0.880325,4.131517,1.606732


Separando os dados

In [30]:
X = df_vazao[['hora',	'dia_semana',	'vazao_15min',	'vazao_1h',	'vazao_24h'] ]
y = df_vazao['vazao']

# Normalizando os dados

In [31]:
from sklearn.preprocessing import StandardScaler

In [32]:
scaled_x = StandardScaler()
scaled_y = StandardScaler()
X_scaled = scaled_x.fit_transform(X)
y_scaled = scaled_y.fit_transform(y.values.reshape(-1,1))

# Separando os dados entre treino e teste

In [33]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Treinando o modelo utilizando uma rede neural

In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu', input_shape=[5]),
    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.Dense(units=1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
train = model.fit(X_train, y_train, epochs=50, validation_split=0.2, batch_size=32, verbose=1)

Epoch 1/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6618 - val_loss: 0.0545
Epoch 2/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0486 - val_loss: 0.0420
Epoch 3/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0419 - val_loss: 0.0357
Epoch 4/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0375 - val_loss: 0.0322
Epoch 5/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0343 - val_loss: 0.0299
Epoch 6/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0324 - val_loss: 0.0288
Epoch 7/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0308 - val_loss: 0.0292
Epoch 8/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0305 - val_loss: 0.0274
Epoch 9/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0295 - val_loss: 0.0269
Epoch 10/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0287 - val_loss: 0.0264
Epoch 11/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0282 - val_loss: 0.0272
Epoch 12/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

# Desempenho

In [37]:
print("\nAvaliando o modelo final...")
loss = model.evaluate(X_test, y_test)
print(f"Loss (MSE) no conjunto de teste: {loss}")


Avaliando o modelo final...
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0143
Loss (MSE) no conjunto de teste: 0.013305660337209702


# Otimizando o modelo

In [38]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

Saved artifact at '/tmp/tmp1mbo177a'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  137605390797136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137605409645264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137605390797904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137605390798288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137605390797712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137605390795984: TensorSpec(shape=(), dtype=tf.resource, name=None)


# Salvando o modelo otimizado em um arquivo

In [39]:
with open('modelo_vazao.tflite', 'wb') as f:
    f.write(tflite_model)